# Segmenter_Locations_shared. April, 2024.
Cell definitions are dictated by convenience in execution

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout 
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import concatenate
from test_utils import summary   #print(tf.__version__)
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import imageio.v2 as imageio
import matplotlib.pyplot as plt
%matplotlib inline
import sys
import statistics
print(sys.version)  #Python version

3.11.4 | packaged by conda-forge | (main, Jun 10 2023, 17:59:51) [MSC v.1935 64 bit (AMD64)]


In [2]:
path = ''
image_path = os.path.join(path, 'C:/Users/erios/EM_Images/split_Penn_train/')
mask_path = os.path.join(path, 'C:/Users/erios/EM_Images/split_Penn_train_locations_labels/')  
image_list_orig = os.listdir(image_path)                  # list within the folder
mask_list_orig = os.listdir(mask_path)                    # list within the folder
image_list = [image_path+i for i in image_list_orig]      # full path and filename
mask_list = [mask_path+i for i in mask_list_orig]         # image_list and mask_list are class lists and have no dtype
image_filenames = tf.constant(image_list)                 # tensor, dtype = string
mask_filenames = tf.constant(mask_list)

dataset = tf.data.Dataset.from_tensor_slices((image_filenames, mask_filenames))

## Utilities

In [3]:
# Preprocess, Data Augmentation
seed = (1, 2)
ran = tf.keras.layers.RandomContrast(0.4)       # RandomContrast of 0.4. Less than for granules.  Must check.
def process_path(image_path, mask_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_png(img, channels=1)
    img = tf.image.convert_image_dtype(img, tf.float32)
    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=0, dtype=tf.dtypes.uint16)
    return img, mask

def preprocess(image, mask):
    global seed
    seed = (seed[0] + random.randint(0, 10000), seed[1] + random.randint(0, 10000))
    #changes the seed randomly
    
    input_image = tf.image.resize(image, (1024,1024), method='nearest')
    input_mask = tf.image.resize(mask, (1024,1024), method='nearest')
    input_image = tf.image.stateless_random_flip_left_right(input_image, seed)
    input_mask = tf.image.stateless_random_flip_left_right(input_mask, seed)
    input_image = tf.image.stateless_random_flip_up_down(input_image, seed)
    input_mask = tf.image.stateless_random_flip_up_down(input_mask, seed)
    input_image = ran(input_image)
    input_image = tf.image.random_brightness(input_image, 0.25)  # RandomBrightness 0.25.  Same as granules. Must check.

    return input_image, input_mask

# prediction from model outputs
def create_mask(pred_mask):
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask[0]
    
def show_predictions(dataset=None, num=2):
    """
    Displays the first image of each of the num batches
    """
    if dataset:
        for image, mask in dataset.take(num):
            pred_mask = unet_loc.predict(image)                           # model name changed to unet_loc
            display([image[0], mask[0], create_mask(pred_mask)])
    else:
        display([sample_image, sample_mask,
            create_mask(unet_loc.predict(sample_image[tf.newaxis, ...]))])

# utility for display. image tensors must be rank 3
def display(display_list):
    plt.figure(figsize=(15, 15))
    title = ['Input Image', 'True Mask', 'Predicted Mask']
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()

## Modules

In [4]:

def conv_block(inputs=None, n_filters=None, dropout_prob=0, max_pooling=True):
    """
    Convolutional downsampling block
    Arguments:
        inputs -- Input tensor
        n_filters -- Number of filters for the convolutional layers
        dropout_prob -- Dropout probability
        max_pooling -- Use MaxPooling2D to reduce the spatial dimensions of the output volume
    Returns: 
        next_layer, skip_connection --  Next layer and skip connection outputs
    """
    conv = Conv2D(n_filters, 3, activation='relu', padding='same',
                  kernel_initializer='he_normal')(inputs)
    conv = Conv2D(n_filters, 3, activation='relu', padding='same',
                  kernel_initializer='he_normal')(conv)
    if dropout_prob > 0:
        conv = Dropout(dropout_prob)(conv)
    if max_pooling:
        next_layer = MaxPooling2D(2,strides=2)(conv)
    else:
        next_layer = conv
    skip_connection = conv
    
    return next_layer, skip_connection

def upsampling_block(expansive_input, contractive_input, n_filters=None):
    """
    Convolutional upsampling block
    Returns: 
        conv -- Tensor output
    """
    up = Conv2DTranspose(n_filters,3,strides=2,padding='same')(expansive_input)
    merge = concatenate([up, contractive_input], axis=3)
    conv = Conv2D(n_filters, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge)
    conv = Conv2D(n_filters, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv)
    
    return conv

def unet_model7(input_size=(1024, 1024, 1), n_filters=4, n_classes=7):
    """
    Function name changed to unet_model7
    Returns: 
    """
    inputs = Input(input_size)
    # Contracting Path (encoding)
    cblock0 = conv_block(inputs, 8)
    cblock1 = conv_block(cblock0[0], 8)
    cblock2 = conv_block(cblock1[0], 16)
    cblock3 = conv_block(cblock2[0], 32)
    cblock4 = conv_block(cblock3[0], 64)
    cblock5 = conv_block(cblock4[0], 128)
    cblock6 = conv_block(cblock5[0], 256, 0.3) # Include a dropout_prob of 0.3 for this layer
    cblock7 = conv_block(cblock6[0], 512, 0.3, max_pooling=False) 
    # Expanding Path (decoding)
    ublock8 = upsampling_block(cblock7[0], cblock6[1], n_filters * 64)
    ublock9 = upsampling_block(ublock8, cblock5[1],  128)
    ublock10 = upsampling_block(ublock9, cblock4[1],  64)
    ublock11 = upsampling_block(ublock10, cblock3[1],  32)
    ublock12 = upsampling_block(ublock11, cblock2[1],  16)
    ublock13 = upsampling_block(ublock12, cblock1[1],  8)
    ublock14 = upsampling_block(ublock13, cblock0[1],  8)
    conv9 = Conv2D(n_filters, 3, activation='relu', padding='same', kernel_initializer='he_normal')(ublock14)
    conv10 = Conv2D(n_classes, 1, padding='same')(conv9)
    
    model = tf.keras.Model(inputs=inputs, outputs=conv10)
    return model

## Model definition, compilation and loading of trained weights

In [5]:
# segmenter location's model is unet_loc
img_height = 1024
img_width = 1024
num_channels = 1
 
unet_loc = unet_model7((img_height, img_width, num_channels))
unet_loc.summary()     # generates summary listed below this code

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1024, 1024, 1)]      0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 1024, 1024, 8)        80        ['input_1[0][0]']             
                                                                                                  
 conv2d_1 (Conv2D)           (None, 1024, 1024, 8)        584       ['conv2d[0][0]']              
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 512, 512, 8)          0         ['conv2d_1[0][0]']            
 D)                                                                                           

In [6]:
# uses Nadam optimizer.  When starting with highly trained weights, use next cell (Adam)
unet_loc.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=2e-3),    # ATTENTION NAdam
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              # metrics=['accuracy'])
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name= "Sparse Categorical Accuracy")])
 

In [22]:
# uses Adam optimizer.
unet_loc.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),   # final stages at 8e-5
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              # metrics=['accuracy'])
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name= "Sparse Categorical Accuracy")])
 

## Model training

In [24]:
coarse_tuning_counter = 0
acc = [0.]  
loss = [0.]  

In [ ]:
# main training
checkpoint_filepath = 'C:/Users/erios/checkpoints/checkpoint_unet_loc_paul2b'  #2b is for use with 6 labels
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,  save_weights_only=True, monitor="Sparse Categorical Accuracy",
    mode='max', save_best_only=True)
image_ds = dataset.map(process_path)                       
processed_image_ds = image_ds.map(preprocess)              
#BUFFER_SIZE = 147
#BATCH_SIZE = 9
BUFFER_SIZE = 17
BATCH_SIZE = 3
train_dataset = processed_image_ds.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
if coarse_tuning_counter == 0:
    initial_epochs = 4         # use in general 5
    total_epochs = initial_epochs
    history = unet_loc.fit(train_dataset, epochs=initial_epochs)
else:
    more_epochs = 20
    total_epochs =  total_epochs + more_epochs
    history = unet_loc.fit(train_dataset, epochs=total_epochs, 
                           initial_epoch=history.epoch[-1], callbacks=[model_checkpoint_callback])
coarse_tuning_counter += 1    
# update history
acc += history.history['Sparse Categorical Accuracy']
loss += history.history['loss']# plot evolution of accuracy and loss

In [ ]:
# update history
#acc += history.history['Sparse Categorical Accuracy']
#loss += history.history['loss']# plot evolution of accuracy and loss
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Sparse Categorical Accuracy')
plt.title('Training Accuracy')
plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.legend(loc='upper right')
plt.ylabel('Sparse Categorical Cross Entropy')
plt.title('Training Loss')
plt.xlabel('epoch')
plt.ylim([0,2])
plt.show()

## Prediction on a "validation" or "test" set. Assumes a ...val_labels folder

In [14]:
# this is a "test" dataset --will not be used in training--  
path= ''
# image_path_val = os.path.join(path, 'C:/Users/erios/EM_Images/split_random_granules_val/')
# mask_path_val = os.path.join(path, 'C:/Users/erios/EM_Images/split_random_granules_val_labels/')
image_path_val = os.path.join(path, 'C:/Users/erios/EM_Images/split_randomized_renamed_ping_val/')
mask_path_val = os.path.join(path, 'C:/Users/erios/EM_Images/split_r_labels_6_val/')
image_list_orig_val = os.listdir(image_path_val)   # a list of image file names
mask_list_orig_val = os.listdir(mask_path_val)
image_list_val = [image_path_val+i for i in image_list_orig_val]   # a list of image file names that includes the ful path
mask_list_val = [mask_path_val+i for i in mask_list_orig_val]   # print(image_list_val[0:1])
image_filenames_val = tf.constant(image_list_val)   #tensor, dtype = string
mask_filenames_val = tf.constant(mask_list_val)

dataset_val = tf.data.Dataset.from_tensor_slices((image_filenames_val, mask_filenames_val))   #a dataset of filenames

In [15]:
# validation .  Preprocess does not include data augmentation 
def process_path(image_path, mask_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_png(img, channels=1)
    img = tf.image.convert_image_dtype(img, tf.float32)
    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=0, dtype=tf.dtypes.uint16)
    return img, mask

def preprocess_val(image, mask):
    input_image = tf.image.resize(image, (1024,1024), method='nearest')
    input_mask = tf.image.resize(mask, (1024,1024), method='nearest')
    return input_image, input_mask

image_ds_val = dataset_val.map(process_path)
processed_image_ds_val = image_ds_val.map(preprocess_val)
val_dataset = processed_image_ds_val.cache().batch(1)    #either way of batching works

## Prediction and output of images as png files

In [ ]:
probs_val = unet_loc.predict(val_dataset)   #makes an np probability array  rank 4
n = probs_val.shape[0]                  # the number of images in the val dataset
probs1_val =  np.vsplit(probs_val, n)  #makes a list of n probability arrays (1024, 1024, 2)
pred_m_l_val= [np.zeros((1024,1024,2),dtype=float) for i in range(n)]   #initialization of a list of np.arrays
pred_val_acc_list= [0.0 for i in range(n)]
for i in range (n):
    pred_m_l_val[i] = create_mask((probs1_val[i]))      #list gets populated by prediction masks
# now will write prediction masks to ping files, with same names as original image files in val folder
DIR = "C:/Users/erios/EM_Images/split_r_labels_6_val_preds/"
for i in range(n):
    name=DIR + str(image_list_orig_val[i])            # list of original image file names
    mask = tf.cast(pred_m_l_val[i],tf.uint16)         #  had to recast numpy prediction masks as uint16 (from int32)
    pred_png = tf.image.encode_png(mask)              #  otherwise, the encoder would not work
    with open(name, 'wb') as f:
        f.write(pred_png.numpy())                     #  works well

## Calculates accuracies for the entire val set and outputs them in a text file   

In [17]:
m = tf.keras.metrics.Accuracy()
true_m_l_val= [np.zeros((1024,1024,2),dtype=float) for i in range(n)]   #initialization of a list of np.arrays true masks
count = 0
for image, mask in val_dataset:
        true_m_l_val[count] = mask[0]                                   # squeezes the first dimension; needed for accuracy calculation
        count=count+1
for i in range(n):
    m.update_state([pred_m_l_val[i]],[ true_m_l_val[i]])
    pred_val_acc_list[i] = m.result().numpy()                           # the accuracy function works in roundabout way
accuracy_out = DIR+"accuracies.txt"
f = open(accuracy_out, "w")
for i in range(n):
    f.write("\n"+str(i)+"    "+str(pred_val_acc_list[i]) )               # a two-column text output
f.close()
#from statistics import mean
print(pred_val_acc_list,sum(pred_val_acc_list)/14.)
print(statistics.mean(pred_val_acc_list[0:7]), statistics.stdev(pred_val_acc_list[0:7]))
# Outputs loss and accuracy lists to text files in DIR.  Could change DIR
loss_out = DIR+"loss_vs_epoch.txt"                      # defines output file name
f = open(loss_out, "w")                                 # logical name of output file
for i in range(25):
    f.write("\n"+str(i)+"    "+str(loss[i]) )               # a two-column text output
f.close()
#probs = unet.predict(train_dataset)   #makes an np array (n_elements in train_dataset, 1024, 1024, 2)
probs1 =  np.vsplit(probs, 64)  #makes a list of arrays (1024, 1024, 2)
pred_m_l= [np.zeros((1024,1024,2),dtype=float) for i in range(64)]   #initialization of a list of np.arrays
for i in range (64):
    pred_m_l[i] = create_mask((probs1[i]))      #list gets populated
# The above list can be written to files as with the val set or used for calculation of metrics

# PRODUCTION

In [ ]:
"""a production run requires loading libraries, defining the input dataset, assembling and compiling the model, defining utilities 
(display_prod, create_mask, process_path, preprocess_prod, show_predictions_prod, and probs_prod, which performs the prediction and outputs the ping predi
predicted masks.  # preparing a production dataset.  No masks. """
path= ''
image_path_prod = os.path.join(path, 'C:/Users/erios/EM_Images/037_split_for_segmentation/')
image_list_orig_prod = os.listdir(image_path_prod)   # a list of image file names
image_list_prod = [image_path_prod+i for i in image_list_orig_prod]   # a list of image file names that includes the full path
image_filenames_prod = tf.constant(image_list_prod)   #tensor, dtype = string

dataset_prod = tf.data.Dataset.from_tensor_slices((image_filenames_prod))   #a dataset of filenames
print(len(image_list_prod))

In [7]:
# prediction from model outputs
def create_mask(pred_mask):
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask[0]
    # Production .  Preprocess does not include data augmentation 
def process_path(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_png(img, channels=1)
    img = tf.image.convert_image_dtype(img, tf.float32)

    return img

def preprocess_prod(image):
    input_image = tf.image.resize(image, (1024,1024), method='nearest')
    return input_image

image_ds_prod = dataset_prod.map(process_path)
processed_image_ds_prod = image_ds_prod.map(preprocess_prod)
#prod_dataset is the production dataset, batched and cached for application of prediction. Not to be confused with dataset_prod
prod_dataset = processed_image_ds_prod.cache().batch(20)    #either way of batching works

In [ ]:
# Production
probs_prod = unet_loc.predict(prod_dataset)   #  PREDICTS! makes an np probability array  rank 4
n = probs_prod.shape[0]                  # the number of images in the prod dataset
probs1_prod =  np.vsplit(probs_prod, n)  #makes a list of n probability arrays (1024, 1024, 2)
pred_m_l_prod= [np.zeros((1024,1024,2),dtype=float) for i in range(n)]   #initialization of a list of np.arrays
pred_prod_acc_list= [0.0 for i in range(n)]
for i in range (n):
    pred_m_l_prod[i] = create_mask((probs1_prod[i]))      #list gets populated by prediction masks
#display([pred_m_l_prod[4]])   #nice check
# now will write prediction masks to ping files, with same names as original image files in val folder
DIR = "C:/Users/erios/EM_Images/037_split_for_segmentation_loc_preds/"  # make sure folder is present
for i in range(n):
    name=DIR + str(image_list_orig_prod[i])            # list of original image file names
    mask = tf.cast(pred_m_l_prod[i],tf.uint16)         #  had to recast numpy prediction masks as uint16 (from int32)
    pred_png = tf.image.encode_png(mask)              #  otherwise, the encoder would not work
    with open(name, 'wb') as f:
        f.write(pred_png.numpy())                     #  works well

## Studying Metrics

In [ ]:
m = tf.keras.metrics.Accuracy()
m.update_state([sample_mask],[ predicted_mask])
m.result().numpy()

In [ ]:
mi = tf.keras.metrics.IoU(num_classes=2, target_class_ids=[1])  IoU depends on the target class
mi.update_state([sample_mask],[ predicted_mask])
mi.result().numpy()

In [ ]:
mi = tf.keras.metrics.IoU(num_classes=2, target_class_ids=[0])
mi.update_state([sample_mask],[ predicted_mask])
mi.result().numpy()